## <b> SQLite Database </b>

In [1]:
import sqlite3

In [2]:
# create a connection to SQLite DB
conn = sqlite3.connect("sample.db")

In [3]:
# create a cursor to help us execute the SQL commands
cursor = conn.cursor()

In [4]:
# create a table
# stocks : stock_code, stock_name

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS stocks(
      stock_code INTEGER PRIMARY KEY,
      stock_name TEXT NOT NULL
    )
    """
)

In [5]:
# insert some data
cursor.execute(
    "INSERT INTO stocks (stock_name) VALUES (?)", ('TESLA',)
)

In [6]:
cursor.execute(
    "INSERT INTO stocks (stock_name) VALUES (?)", ('Microsoft',)
)

In [7]:
# select records
cursor.execute("SELECT * FROM stocks")

In [8]:
rows = cursor.fetchall()
rows

[(1, 'TESLA'), (2, 'Microsoft')]

In [9]:
conn.commit()

In [10]:
conn.close()

### <b> Using SQLite as vector storage </b>

In [11]:
# vector = array of numbers -> numpy arrays -> [1.2, 2.5, 3.7]
# store the information into a bytes format

In [12]:
import numpy as np

In [13]:
conn = sqlite3.connect("sample_vector.db")

In [14]:
cursor = conn.cursor()

In [15]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS vectors (
      vector_id INTEGER PRIMARY KEY,
      vector BLOB NOT NULL
    )
    """
)

In [16]:
vect_tsla = np.array([1.3, 3.5, 2.2, 0.9])
vect_msft = np.array([2.8, 1.6, 3.8, 2.2])

In [17]:
type(vect_tsla)

numpy.ndarray

In [18]:
vect_tsla.tobytes()

b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'

In [19]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect_tsla.tobytes()),))

In [20]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect_msft.tobytes()),))

In [21]:
cursor.execute("SELECT * FROM vectors")

In [22]:
rows = cursor.fetchall()
rows

[(1,
  b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'),
 (2,
  b'ffffff\x06@\x9a\x99\x99\x99\x99\x99\xf9?ffffff\x0e@\x9a\x99\x99\x99\x99\x99\x01@')]

In [23]:
# Deserialization

In [24]:
# np.frombuffer

In [25]:
v = np.frombuffer(rows[0][1], dtype=np.float64)
v

array([1.3, 3.5, 2.2, 0.9])

In [26]:
vectors = []
for row in rows:
  v = np.frombuffer(row[1], dtype=np.float64)
  vectors.append(v)

In [27]:
vectors

[array([1.3, 3.5, 2.2, 0.9]), array([2.8, 1.6, 3.8, 2.2])]

In [28]:
# Find nearest vector

In [29]:
q_vector = np.array([2.5, 1.2, 3.5, 5.5])

In [30]:
cursor.execute("""
SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC
""", (sqlite3.Binary(q_vector.tobytes()),)
)

In [31]:
res = cursor.fetchone()

In [32]:
res

(b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?',)

In [33]:
np.frombuffer(res[0], dtype=np.float64)

array([1.3, 3.5, 2.2, 0.9])